In [1]:
from IPython.display import HTML, display

html = """
<style>
  .ttt-container {
    font-family: Inter, Roboto, Arial, sans-serif;
    display:flex;
    flex-direction:column;
    align-items:center;
    gap:16px;
    margin:16px 0;
  }
  .board {
    display:grid;
    grid-template-columns: repeat(3, 100px);
    grid-template-rows: repeat(3, 100px);
    gap:8px;
  }
  .cell {
    width:100px;
    height:100px;
    display:flex;
    align-items:center;
    justify-content:center;
    font-size:48px;
    font-weight:700;
    background: linear-gradient(180deg, #ffffff, #f2f4f8);
    border-radius:12px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.08), inset 0 -2px 0 rgba(0,0,0,0.03);
    cursor:pointer;
    user-select:none;
    transition: transform .08s ease, box-shadow .08s ease;
  }
  .cell:active { transform: scale(0.98); }
  .cell.disabled { cursor: default; opacity:0.9; }
  .status {
    font-size:18px;
    padding:8px 12px;
    border-radius:10px;
    background:#0f172a;
    color: white;
    box-shadow: 0 6px 18px rgba(2,6,23,0.35);
  }
  .controls { display:flex; gap:8px; align-items:center; }
  button.btn {
    background:#0ea5a4;
    border:none;
    color:white;
    padding:8px 12px;
    border-radius:8px;
    cursor:pointer;
    font-weight:600;
  }
  button.secondary {
    background:#64748b;
  }
  .scores { display:flex; gap:12px; }
  .score-card {
    padding:8px 12px;
    border-radius:8px;
    background:#ffffff;
    box-shadow: 0 4px 12px rgba(0,0,0,0.06);
    min-width:110px;
    text-align:center;
  }
  .win-line {
    position: absolute;
    pointer-events:none;
    transition: opacity .25s ease;
  }
  .board-wrap { position:relative; }
  .small { font-size:14px; color:#334155; }
  .footer { font-size:13px; color:#475569; }
</style>

<div class="ttt-container">
  <div class="status" id="status">Turno: <strong id="turn">X</strong></div>

  <div class="board-wrap">
    <div class="board" id="board">
      <div class="cell" data-index="0"></div>
      <div class="cell" data-index="1"></div>
      <div class="cell" data-index="2"></div>
      <div class="cell" data-index="3"></div>
      <div class="cell" data-index="4"></div>
      <div class="cell" data-index="5"></div>
      <div class="cell" data-index="6"></div>
      <div class="cell" data-index="7"></div>
      <div class="cell" data-index="8"></div>
    </div>
    <!-- SVG line for visual win (created dynamically) -->
    <svg id="winSvg" class="win-line" width="340" height="340" style="top:-8px; left:-8px;"></svg>
  </div>

  <div class="controls">
    <div class="scores">
      <div class="score-card">
        <div class="small">Jugador X</div>
        <div id="scoreX">0</div>
      </div>
      <div class="score-card">
        <div class="small">Empates</div>
        <div id="scoreDraw">0</div>
      </div>
      <div class="score-card">
        <div class="small">Jugador O</div>
        <div id="scoreO">0</div>
      </div>
    </div>

    <div style="width:16px"></div>

    <button class="btn" id="restart">Reiniciar partida</button>
    <button class="btn secondary" id="resetScores">Reset scores</button>
  </div>

  <div class="footer small">Dos jugadores en la misma máquina — clic para jugar. Código simple listo para modificar.</div>
</div>

<script>
(function(){
  const boardEl = document.getElementById('board');
  const cells = Array.from(boardEl.querySelectorAll('.cell'));
  const turnEl = document.getElementById('turn');
  const statusEl = document.getElementById('status');
  const svg = document.getElementById('winSvg');

  const scoreXEl = document.getElementById('scoreX');
  const scoreOEl = document.getElementById('scoreO');
  const scoreDrawEl = document.getElementById('scoreDraw');

  const restartBtn = document.getElementById('restart');
  const resetScoresBtn = document.getElementById('resetScores');

  let board = Array(9).fill(null);
  let turn = 'X';
  let playing = true;
  let scores = { X: 0, O: 0, D: 0 };

  // winning lines with cell indices and SVG line coordinates (approx)
  const winLines = [
    { idx: [0,1,2], line: {x1:50,y1:50,x2:250,y2:50} },
    { idx: [3,4,5], line: {x1:50,y1:150,x2:250,y2:150} },
    { idx: [6,7,8], line: {x1:50,y1:250,x2:250,y2:250} },
    { idx: [0,3,6], line: {x1:50,y1:50,x2:50,y2:250} },
    { idx: [1,4,7], line: {x1:150,y1:50,x2:150,y2:250} },
    { idx: [2,5,8], line: {x1:250,y1:50,x2:250,y2:250} },
    { idx: [0,4,8], line: {x1:50,y1:50,x2:250,y2:250} },
    { idx: [2,4,6], line: {x1:250,y1:50,x2:50,y2:250} },
  ];

  function render() {
    cells.forEach((c,i) => {
      c.textContent = board[i] || '';
      if (!playing) c.classList.add('disabled'); else c.classList.remove('disabled');
    });
    turnEl.textContent = turn;
  }

  function checkWin(bd) {
    for (const w of winLines) {
      const [a,b,c] = w.idx;
      if (bd[a] && bd[a] === bd[b] && bd[a] === bd[c]) {
        return { winner: bd[a], line: w.line };
      }
    }
    if (bd.every(Boolean)) return { winner: 'D' }; // draw
    return null;
  }

  function showWinLine(lineObj) {
    svg.innerHTML = '';
    const line = document.createElementNS("http://www.w3.org/2000/svg","line");
    line.setAttribute('x1', lineObj.x1);
    line.setAttribute('y1', lineObj.y1);
    line.setAttribute('x2', lineObj.x2);
    line.setAttribute('y2', lineObj.y2);
    line.setAttribute('stroke','rgba(14,165,164,0.95)');
    line.setAttribute('stroke-width','10');
    line.setAttribute('stroke-linecap','round');
    line.setAttribute('opacity','0');
    svg.appendChild(line);
    // animate appearance
    setTimeout(()=> line.setAttribute('opacity','1'), 10);
  }

  function onCellClick(e) {
    if (!playing) return;
    const idx = Number(e.currentTarget.dataset.index);
    if (board[idx]) return;
    board[idx] = turn;
    const result = checkWin(board);
    if (result) {
      playing = false;
      if (result.winner === 'D') {
        statusEl.textContent = "Empate ";
        scores.D += 1;
        scoreDrawEl.textContent = scores.D;
      } else {
        statusEl.textContent = `¡Victoria! Gana ${result.winner}`;
        scores[result.winner] += 1;
        if (result.winner === 'X') scoreXEl.textContent = scores.X;
        else scoreOEl.textContent = scores.O;
        showWinLine(result.line);
      }
    } else {
      // swap turn
      turn = (turn === 'X') ? 'O' : 'X';
      statusEl.innerHTML = 'Turno: <strong id="turn">'+turn+'</strong>';
      // rebind turnEl reference
      // (not strictly necessary but keep display correct)
    }
    render();
  }

  function resetBoard(nextStarter = null) {
    board = Array(9).fill(null);
    playing = true;
    svg.innerHTML = '';
    if (nextStarter) turn = nextStarter;
    statusEl.innerHTML = 'Turno: <strong id="turn">'+turn+'</strong>';
    render();
  }

  // attach listeners
  cells.forEach(c => c.addEventListener('click', onCellClick));
  restartBtn.addEventListener('click', ()=> {
    // alternate starter to keep variety
    turn = (turn === 'X') ? 'O' : 'X';
    resetBoard(turn);
  });
  resetScoresBtn.addEventListener('click', ()=> {
    scores = { X:0, O:0, D:0 };
    scoreXEl.textContent = '0';
    scoreOEl.textContent = '0';
    scoreDrawEl.textContent = '0';
    resetBoard('X');
  });

  // initial render
  resetBoard('X');

  // keyboard support: numbers 1-9 map to cells (numpad style)
  document.addEventListener('keydown', (ev)=>{
    if (!playing) return;
    const map = {
      '1':6,'2':7,'3':8,
      '4':3,'5':4,'6':5,
      '7':0,'8':1,'9':2
    };
    const key = ev.key;
    if (map.hasOwnProperty(key)) {
      const idx = map[key];
      const target = cells[idx];
      if (target) target.click();
    }
  });
})();
</script>
"""

display(HTML(html))


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output


board = [" "] * 9
turn = "X"

buttons = [widgets.Button(description=" ", layout=widgets.Layout(width="60px", height="60px")) for _ in range(9)]
status = widgets.Label(value=f"Turno: {turn}")

def check_winner(b):
    combos = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
    for x,y,z in combos:
        if board[x] == board[y] == board[z] != " ":
            return board[x]
    if " " not in board:
        return "Empate"
    return None

def on_click(b):
    global turn, board
    idx = buttons.index(b)
    if board[idx] == " ":
        board[idx] = turn
        b.description = turn
        winner = check_winner(board)
        if winner:
            if winner == "Empate":
                status.value = "¡Empate!"
            else:
                status.value = f"¡Ganó {winner}! "
            for btn in buttons:
                btn.disabled = True
        else:
            turn = "O" if turn == "X" else "X"
            status.value = f"Turno: {turn}"


for btn in buttons:
    btn.on_click(on_click)


def restart(_):
    global board, turn
    board = [" "] * 9
    turn = "X"
    status.value = f"Turno: {turn}"
    for btn in buttons:
        btn.description = " "
        btn.disabled = False

restart_btn = widgets.Button(description="Reiniciar partida", button_style="success")
restart_btn.on_click(restart)


grid = widgets.GridBox(buttons, layout=widgets.Layout(grid_template_columns="repeat(3, 60px)"))
display(status, grid, restart_btn)


Label(value='Turno: X')

GridBox(children=(Button(description=' ', layout=Layout(height='60px', width='60px'), style=ButtonStyle()), Bu…

Button(button_style='success', description='Reiniciar partida', style=ButtonStyle())